In [16]:
import pandas as pd


df = pd.read_csv(r"C:\Users\HP\Downloads\Canteen Recommendation System\ML\Data\raw\mock_canteen_orders.csv", parse_dates=["timestamp"])

df.head()


,order_id,user_id,item_id,item_name,category,quantity,total_price,timestamp
0,1,U186,D05,Idli,Breakfast,1,50,2025-10-23 06:12:00
1,2,U163,D03,Tea,Beverage,1,15,2025-10-29 18:34:00
2,3,U099,D10,Shake,Beverage,1,70,2025-10-19 17:55:00
3,4,U179,D03,Tea,Beverage,1,15,2025-10-25 06:13:00
4,5,U135,D02,Paneer Roll,Lunch,1,120,2025-10-12 06:10:00


In [17]:
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     1200 non-null   int64         
 1   user_id      1200 non-null   object        
 2   item_id      1200 non-null   object        
 3   item_name    1200 non-null   object        
 4   category     1200 non-null   object        
 5   quantity     1200 non-null   int64         
 6   total_price  1200 non-null   int64         
 7   timestamp    1200 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 75.1+ KB


order_id       0
user_id        0
item_id        0
item_name      0
category       0
quantity       0
total_price    0
timestamp      0
dtype: int64

In [18]:
popular = (
    df["item_name"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "dish_name", "item_name": "order_count"})
)
popular.head(10)


,order_count,count
0,Tea,249
1,Paneer Roll,192
2,Veg Thali,146
3,Samosa,115
4,Burger,104
5,Coffee,94
6,Poha,93
7,Idli,91
8,Shake,58
9,Paratha,58


In [19]:
from datetime import datetime, timedelta

# Popular in the last 7 days
recent = df[df["timestamp"] > (df["timestamp"].max() - timedelta(days=7))]
recent_popular = recent["item_name"].value_counts().head(5)
print("Trending This Week:\n", recent_popular)

# Popular by meal period
def get_meal_period(hour):
    if 6 <= hour < 11:
        return "Breakfast"
    elif 11 <= hour < 16:
        return "Lunch"
    else:
        return "Dinner"

df["meal_period"] = df["timestamp"].dt.hour.apply(get_meal_period)
meal_popular = (
    df.groupby("meal_period")["item_name"]
    .value_counts()
    .groupby(level=0)
    .head(3)
)
print(meal_popular)


Trending This Week:
 item_name
Tea            52
Paneer Roll    37
Veg Thali      33
Samosa         23
Poha           22
Name: count, dtype: int64
meal_period  item_name  
Breakfast    Tea            78
             Paneer Roll    60
             Veg Thali      46
Dinner       Tea            89
             Paneer Roll    80
             Veg Thali      64
Lunch        Tea            82
             Paneer Roll    52
             Burger         41
Name: count, dtype: int64


In [20]:
def get_recommendations(df, mode="popular", top_n=5):
    if mode == "popular":
        data = (
            df.groupby("item_name")["quantity"]
              .sum()
              .reset_index()
              .sort_values(by="quantity", ascending=False)
        )
    elif mode == "trending":
        recent_period = df["timestamp"].max() - pd.Timedelta(days=7)
        data = (
            df[df["timestamp"] >= recent_period]
              .groupby("item_name")["quantity"]
              .sum()
              .reset_index()
              .sort_values(by="quantity", ascending=False)
        )
    else:
        raise ValueError("Mode must be 'popular' or 'trending'")
    return data.head(top_n)["item_name"].tolist()

# Try both modes
print("Top popular dishes:", get_recommendations(df, "popular", 5))
print("Top trending dishes:", get_recommendations(df, "trending", 5))


Top popular dishes: ['Tea', 'Paneer Roll', 'Veg Thali', 'Samosa', 'Burger']
Top trending dishes: ['Tea', 'Paneer Roll', 'Veg Thali', 'Poha', 'Samosa']


In [21]:
top5 = popular.head(5)
top5.to_json(r"C:\Users\HP\Downloads\Canteen Recommendation System\ML\Data\top_dishes.json", orient="records", indent=2)
print("✅ top_dishes.json saved!")


✅ top_dishes.json saved!
